In [ ]:
import twitter
import json
import time
import psycopg2 as pg2
import numpy as np
import pandas as pd

from datetime import datetime
from psycopg2.extras import RealDictCursor, Json

In [ ]:
%run sql_test.py

In [ ]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

In [ ]:
twitter_keys = {
    'consumer_key':        'OoTmt8ytXol9yRJxLvBbYER5e',
    'consumer_secret':     'wHEGZ4juW2AIUU1GFp32oA8vGtre8Se5hSPp5DfOmeNJ2a3Ld7',
    'access_token_key':    '1080874755215872001-9F7ZdDBK2b9hgBKjSTiTQXIWY2XdyM',
    'access_token_secret': 'HIMPWRosWf2Wlu73wEPWjsmjobO3FvtRaa3MneDW0wT3q'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
#     sleep_on_rate_limit=True
)

In [ ]:
results = api.GetSearch(
            geocode='35.2271,-80.8431,2mi',
            term = 'timberlake%20justin%20concert%20show%' 
            return_json = True
        )

In [ ]:
print(api.VerifyCredentials())

In [ ]:
def streamTweets(city, geocode, before, count, sql_db):
    for i in range(1,8):
        year, month, day = before.split('-')
#         day = int(day)
#         day-=1
#         day = str(day).zfill(2)
#         date = year + month + day
#         after = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        
        results = api.GetSearch(
            geocode=geocode,
            return_json = True
        )

        insert_entry_json(results['statuses'], sql_db)
        before = after

In [ ]:
def tweet_repeater(city, geocode, start, repeats=100, count=100, sql_db='raw_tweets'):
    for i in range(len(repeats)):
        before = start
        
        streamTweets(city, geocode, before, count)
        time.sleep(1)
        print(f'Loop {i} complete. Raw tweets pushed to {sql_db}.')
    print('All tweets pulled.')

In [ ]:
tweet_repeater(city='cha', geocode='35.2271,-80.8431,2mi', start='2019-01-08', repeats=5000, count=100, sql_db='raw_tweets')